# Exercise - Decision Tree- AAKESH U63411601

This data set for this exercise is from healthcare. It contains data about 10 years of clinical care at 130 US Hospitals. Each row represents a single patient. The columns include the characteristics of deidentified diabetes patients. This is a binary classification task: predict whether a diabetes patient is readmitted to the hospital within 30 days of their discharge (1=Yes, 0=No). This is an important performance metric for hospitals as they try to minimize these types of readmissions.

## Description of Variables

The description of variables are provided in "Healthcare (small) - Data Dictionary.docx"

## Goal

Use the **healthcare_small.csv** data set and build a model to predict **readmitted**. 

# Read and Prepare the Data

In [1]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [2]:
#We will predict the "price" value in the data set:

diabetes = pd.read_csv("healthcare_small.csv")
diabetes.head()

,race,gender,age,admission_type,discharge_disposition,admission_source,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmitted
0,Other,Female,70-80,2,3,1,14,32,3,15,0,0,0,9,1
1,Caucasian,Female,80-90,1,3,5,4,44,0,15,0,0,0,9,0
2,AfricanAmerican,Male,50-60,5,1,1,6,29,1,15,0,0,0,9,1
3,Caucasian,Female,50-60,1,1,6,3,47,0,10,0,0,0,4,0
4,AfricanAmerican,Female,40-50,3,1,1,4,92,0,15,0,0,0,7,0


# Split data (train/test)

In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(diabetes, test_size=0.3)

# Data Prep

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

## Separate the target variable 

In [5]:
train_target = train['readmitted']
test_target = test['readmitted']

train_inputs = train.drop(['readmitted'], axis=1)
test_inputs = test.drop(['readmitted'], axis=1)

## Feature Engineering: Derive a new column

Examples:
- Whether the patient had any emergency visits or not
- Ratio of inpatient visits to outpatient visits
- Ratio of time in hospital to number of medications
- (etc.)

### FORMULA: ratio_medications_to_procedures = num_medications / num_procedures


In [6]:
def new_col(df):
    
    #Create a copy so that we don't overwrite the existing dataframe
    df1 = df.copy()

    # Use the formula, though fill in 0s when the value is 0/0 (because 0/0 generates "nan" values)
    df1['ratio_medications_to_procedures'] = (df1['num_medications']/df1['num_procedures']).fillna(0)

    # Replace the infinity values with 1 (because a value divided by 0 generates infinity)
    df1['ratio_medications_to_procedures'].replace(np.inf, 1, inplace=True)

    return df1[['ratio_medications_to_procedures']]

In [7]:
#Let's test the new function:

# Send the train set to the function we created
new_col(train)

,ratio_medications_to_procedures
4166,19.000000
5546,1.000000
2957,9.500000
6329,20.000000
565,4.000000
...,...
5734,10.333333
5191,1.000000
5390,1.000000
860,2.800000


##  Identify the numeric, binary, and categorical columns

In [8]:
# Let's change some of the categorical variables' data type to object

train_inputs['admission_source'] = train_inputs['admission_source'].astype('object')
train_inputs['admission_type'] = train_inputs['admission_type'].astype('object')
train_inputs['discharge_disposition'] = train_inputs['discharge_disposition'].astype('object')

test_inputs['admission_source'] = test_inputs['admission_source'].astype('object')
test_inputs['admission_type'] = test_inputs['admission_type'].astype('object')
test_inputs['discharge_disposition'] = test_inputs['discharge_disposition'].astype('object')


In [9]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [10]:
numeric_columns

['time_in_hospital',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'number_diagnoses']

In [11]:
categorical_columns

['race',
 'gender',
 'age',
 'admission_type',
 'discharge_disposition',
 'admission_source']

In [12]:
feat_eng_columns = ['num_medications', 'num_procedures']

# Pipeline

In [13]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [14]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [15]:
# Create a pipeline for the transformed column here
my_new_column = Pipeline(steps=[('ratio_medications_to_procedures', FunctionTransformer(new_col)),
                               ('scaler', StandardScaler())])

In [16]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('trans', my_new_column, feat_eng_columns)],   
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [17]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

<6066x65 sparse matrix of type '<class 'numpy.float64'>'
	with 90990 stored elements in Compressed Sparse Row format>

In [18]:
train_x.shape

(6066, 65)

# Tranform: transform() for TEST

In [19]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

<2600x65 sparse matrix of type '<class 'numpy.float64'>'
	with 38997 stored elements in Compressed Sparse Row format>

In [20]:
test_x.shape

(2600, 65)

# Calculate the Baseline

In [32]:
from sklearn.dummy import DummyRegressor

dummy_regr = DummyRegressor(strategy="mean")

dummy_regr.fit(train_x, train_target)

DummyRegressor()

In [33]:
from sklearn.metrics import mean_squared_error

In [34]:
#Baseline Train RMSE
dummy_train_pred = dummy_regr.predict(train_x)

baseline_train_mse = mean_squared_error(train_target, dummy_train_pred)

baseline_train_rmse = np.sqrt(baseline_train_mse)

print('Baseline Train RMSE: {}' .format(baseline_train_rmse))

Baseline Train RMSE: 0.4990179601891472


In [35]:
#Baseline Test RMSE
dummy_test_pred = dummy_regr.predict(test_x)

baseline_test_mse = mean_squared_error (test_target, dummy_test_pred)

baseline_test_rmse = np.sqrt(baseline_test_mse)

print('Baseline Test RMSE: {}' .format(baseline_test_rmse))

Baseline Test RMSE: 0.49869043605518787


# Train a DT model

In [101]:
from sklearn.tree import DecisionTreeClassifier 

tree_clf = DecisionTreeClassifier(criterion='gini', max_depth=10, splitter='random')

tree_clf.fit(train_x, train_target)

DecisionTreeClassifier(max_depth=10, splitter='random')

In [102]:
from sklearn.metrics import accuracy_score

### Calculate the accuracy

In [103]:
#Train accuracy:
train_target_pred = tree_clf.predict(train_x)

print(accuracy_score(train_target, train_target_pred))

0.6486976590834158


In [104]:
#Test accuracy:
test_target_pred = tree_clf.predict(test_x)

print(accuracy_score(test_target, test_target_pred))

0.6023076923076923


### feature Impotance

In [106]:
# There are 62 columns, so there are 62 values for each column's importance
# Values are rounded to three decimals

np.round(tree_clf.feature_importances_,3)

array([0.027, 0.02 , 0.04 , 0.032, 0.023, 0.022, 0.353, 0.024, 0.004,
       0.   , 0.004, 0.009, 0.004, 0.01 , 0.01 , 0.002, 0.   , 0.006,
       0.006, 0.004, 0.006, 0.016, 0.008, 0.002, 0.009, 0.02 , 0.012,
       0.01 , 0.008, 0.   , 0.013, 0.004, 0.   , 0.016, 0.002, 0.033,
       0.003, 0.   , 0.013, 0.   , 0.   , 0.   , 0.049, 0.007, 0.   ,
       0.   , 0.   , 0.   , 0.075, 0.006, 0.   , 0.   , 0.   , 0.   ,
       0.017, 0.   , 0.   , 0.009, 0.009, 0.016, 0.002, 0.   , 0.005,
       0.003, 0.028])

In [108]:
np.round(tree_clf.feature_importances_,3)[-1]

0.028

# Train another DT model (with different parameters)

In [72]:
from sklearn.tree import DecisionTreeClassifier 

tree_clf1 = DecisionTreeClassifier(criterion='gini', max_depth=10, min_samples_leaf = 10, max_features=10)

tree_clf1.fit(train_x, train_target)

DecisionTreeClassifier(max_depth=10, max_features=10, min_samples_leaf=10)

In [73]:
from sklearn.metrics import accuracy_score

### Calculate the accuracy

In [74]:
#Train accuracy:
train_target_pred = tree_clf1.predict(train_x)

print(accuracy_score(train_target, train_target_pred))

0.6348499835146719


In [75]:
#Test accuracy:
test_target_pred = tree_clf1.predict(test_x)

print(accuracy_score(test_target, test_target_pred))

0.5857692307692308


# Optional: try grid search